In [1]:
import numpy as np

In [86]:
class vertice():
    def __init__(self, num=None, L=None, nivel=None, PE=None, PS=None, pai=None):
        self.PE = PE
        self.PS = PS
        self.pai = pai
        self.L = L
        self.nivel = nivel
        self.num = num
        
class aresta():
    def __init__(self, v, w ,cor='255,255,255'):
        self.cor = cor
        self.aresta = (v,w)
        
class grafo():
    def __init__(self, path):
        self.path = path
        self.G = None
        self.n_V = None
        self.lista_adj = {}
        self.vertices = []
        self.arestas = []
        
    
    def matriz(self):
        arq = open(self.path,'r')

        self.G = [linha.rstrip() for linha in arq]
        self.G[0] = int(self.G[0])
        self.n_V = self.G[0]
        self.G.remove(self.n_V)

        for i in range(len(self.G)):
            self.G[i] = self.G[i].split(' ')

            for j in range(len(self.G[i])):
                self.G[i][j] = int(self.G[i][j])

        arq.close()
        self.G = np.array(self.G)
        m = self.G.shape[0]

        for i in range(m):
            for j in range(i,m):
                if self.G[i][j] == 1:
                    self.arestas.append( aresta(i, j) )

        
        print()
        return self.G

    
    def retorna_ind(self,v1,v2):
        arestas_aux = []

        for obj in self.arestas:
            arestas_aux.append([obj.aresta])

        try:
            ind = arestas_aux.index([(v1, v2)]) 
            
        except:
            ind = arestas_aux.index([(v2, v1)])
            
        return ind
        
    def print_matriz_lista(self):
    
        print('Matriz de adjacência: \n')
    
        for i in range(len(self.G)):

            print(f'{i} -> {self.G[i]}')
        
        print('\nLista de adjacência: \n')
        
        for v in self.lista_adj:
            print(f'{v} -> {self.lista_adj[v]}')
    

    def criar_arquivo(self, filename):
        arq = open(f'{filename}.gdf', 'w');
        ordem = self.n_V 

        arq.write('nodedef>name VARCHAR,label VARCHAR\n')
        for i in range(ordem):
            arq.write(f'{i+1},{i+1}\n')

        arq.write('edgedef>node1 VARCHAR,node2 VARCHAR,directed BOOLEAN,color VARCHAR\n')
        for aresta in self.arestas:
            arq.write(f'{aresta.aresta[0] + 1},{aresta.aresta[1] + 1},false,\'{aresta.cor}\'\n')

        arq.close()    
    
    def lista_adj_(self, DFS=True):
        grafo = self.matriz()

        m,n = grafo.shape

        for i in range(m):
            lista = []
            for j in range(n):
                if grafo[i][j] == 1:
                    lista.append(j)

                self.lista_adj[i] = lista
        
        if DFS:
            self.ini_DFS()
    
    def ini_DFS(self):
        for v in self.lista_adj:
            self.vertices.append(vertice(num=v, PE=0, PS=0, pai=None))
            
       
    def DFS(self, root):    
        t = 0
        root = self.vertices[root].num
        t = t + 1 
        
        self.vertices[root].PE = t
        
        for w in self.lista_adj[root]:
            if self.vertices[w].PE == 0:
                ind = self.retorna_ind(root,w)
                self.arestas[ind].cor = '0,0,255'
                self.vertices[w].pai = root
                self.DFS(w)
            
            else:
                if self.vertices[w].PS == 0 and w != self.vertices[root].pai:
                    ind = self.retorna_ind(root,w)
                    self.arestas[ind].cor = '255,0,0'
                
        t = t + 1
        self.vertices[root].PS = t
    
    def BFS(self):
        for v in self.lista_adj:
            self.vertices.append(vertice(num=v, L=0, pai=None))
        
        t = 0
        fila_aux = []
        
        for v in self.lista_adj.keys():
            if self.vertices[v].L == 0:
                self.vertices[v].nivel = 0
                t = t + 1
                self.vertices[v].L = t
                fila_aux.append(v)
                
                while len(fila_aux) != 0:
                    vert = fila_aux[0]
                    fila_aux.pop(0)
                    
                    for w in self.lista_adj[vert]:
                        if self.vertices[w].L == 0:
                            ind = self.retorna_ind(vert,w)
                            self.arestas[ind].cor = '0,0,255'
                            self.vertices[w].pai = vert
                            self.vertices[w].nivel = self.vertices[vert].nivel + 1
                            t = t + 1
                            self.vertices[w].L = t
                            fila_aux.append(w)
                        
                        else:
                            if self.vertices[w].nivel == self.vertices[vert].nivel:
                                if self.vertices[w].pai == self.vertices[vert].pai:
                                    ind = self.retorna_ind(vert,w)
                                    self.arestas[ind].cor = '255,0,0'
                                
                                else:
                                    ind = self.retorna_ind(vert,w)
                                    self.arestas[ind].cor = '255,255,0'
                            else:
                                if self.vertices[w].nivel == self.vertices[vert].nivel + 1:
                                    ind = self.retorna_ind(vert,w)
                                    self.arestas[ind].cor = '0,255,0'
                            
                

In [87]:
g1 = grafo('./in/graph_2')
g1.lista_adj_(DFS=True)
g1.DFS(0)
g1.criar_arquivo('graph_2_dfs')

g2 = grafo('./in/graph_2')
g2.lista_adj_(DFS=False)
g2.BFS()
g2.criar_arquivo('graph_2_bfs')